In [1]:
from oct_Utils import *

In [87]:
root_path = '/home/octusr2/wk/projects/oct/output'
exp_name = '0829/macula_num_r50_380'

In [88]:
df = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/data.csv')
print_df(df)

['image_path', 'source', 'pid', 'dates', 'eye', 'pos', 'num', 'dataset', 'pred']
10634


,image_path,source,pid,dates,eye,pos,num,dataset,pred
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,0,[25.709463 26.031473 25.66152 25.685753 2...
...,...,...,...,...,...,...,...,...,...
10633,20210820-sysu-oct/magic/CRC000589/20210416/OD/...,magic,CRC000589,2021-04-16,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,4,[26.626272 27.131907 26.806307 26.533113...


In [89]:
valid_mask ='''[[0 0 0 0 0 0 0 0 0 0]
                [0 0 0 1 1 1 1 0 0 0]
                [0 0 1 1 1 1 1 1 0 0]
                [0 1 1 1 1 1 1 1 1 0]
                [1 1 1 1 1 1 1 1 1 0]
                [1 1 1 1 1 1 1 1 1 0]
                [0 1 1 1 1 1 1 1 1 0]
                [0 0 1 1 1 1 1 1 0 0]
                [0 0 0 1 1 1 1 0 0 0]
                [0 0 0 0 0 0 0 0 0 0]]'''

od_valid_mask = str_to_np_mat(valid_mask) == 1


def restore_pred_matrix(row):
    result = np.zeros((10, 10), dtype=float)
    result[od_valid_mask] = str_to_np_array(row['pred'])
    if row['eye'] == 'OS':
        result = np.flip(result, axis=1)
    return result

def restore_pred_list(row):
    mat = restore_pred_matrix(row)
    if row['eye'] == 'OS':
        mat = np.flip(mat, axis=1)
    return mat[od_valid_mask].tolist()

def convert_num_to_list(row):
    mat = str_to_np_mat(row['num'])
    if row['eye'] == 'OS':
        mat = np.flip(mat, axis=1)
    return mat[od_valid_mask].tolist()

df['pred_mat'] = df.apply(lambda row: restore_pred_matrix(row), axis=1)
df['pred_array'] = df.apply(lambda row: restore_pred_list(row), axis=1)
df['num_array'] = df.apply(lambda row: convert_num_to_list(row), axis=1)

print_df(df)

['image_path', 'source', 'pid', 'dates', 'eye', 'pos', 'num', 'dataset', 'pred', 'pred_mat', 'pred_array', 'num_array']
10634


,image_path,source,pid,dates,eye,pos,num,dataset,pred,pred_mat,pred_array,num_array
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,0,[25.709463 26.031473 25.66152 25.685753 2...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.709463, 26.031473, 25.66152, 25.685753, 26...","[34.0, 29.0, 31.0, 26.0, 31.0, 32.0, 31.0, 30...."
...,...,...,...,...,...,...,...,...,...,...,...,...
10633,20210820-sysu-oct/magic/CRC000589/20210416/OD/...,magic,CRC000589,2021-04-16,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,4,[26.626272 27.131907 26.806307 26.533113...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[26.626272, 27.131907, 26.806307, 26.533113, 2...","[30.0, 27.0, 28.0, 31.0, 29.0, 32.0, 33.0, 32...."


In [90]:
meta_df = pd.read_csv('/home/octusr2/projects/data_fast/csv/0829.csv')
meta_df = meta_df.drop_duplicates(subset=['image_path'])
meta_df = meta_df[['image_path', 'age', 'floss', 'FP', 'FN']]
meta_df['age'] = pd.to_numeric(meta_df['age'],errors='coerce')
print_df(meta_df)

join_df = df.merge(meta_df, on='image_path')
join_df = join_df.dropna()

print_df(join_df)

['image_path', 'age', 'floss', 'FP', 'FN']
13421


,image_path,age,floss,FP,FN
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CRC-009-143...,NaN,0.0,0.01,0.00
...,...,...,...,...,...
21647,20210820-sysu-oct/magic/CRC000654/20201013/OS/...,59.0,0.0,0.01,0.01


['image_path', 'source', 'pid', 'dates', 'eye', 'pos', 'num', 'dataset', 'pred', 'pred_mat', 'pred_array', 'num_array', 'age', 'floss', 'FP', 'FN']
10538


,image_path,source,pid,dates,eye,pos,num,dataset,pred,pred_mat,pred_array,num_array,age,floss,FP,FN
0,20210730-中山眼科-OCT转VF/CASIA数据汇总-new/CS-199/2019...,CASIA数据汇总-new,CS-199,2019-06-12,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,0,[25.709463 26.031473 25.66152 25.685753 2...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[25.709463, 26.031473, 25.66152, 25.685753, 26...","[34.0, 29.0, 31.0, 26.0, 31.0, 32.0, 31.0, 30....",42.0,0.071429,0.02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10633,20210820-sysu-oct/magic/CRC000589/20210416/OD/...,magic,CRC000589,2021-04-16,OD,macula,[[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [...,4,[26.626272 27.131907 26.806307 26.533113...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[26.626272, 27.131907, 26.806307, 26.533113, 2...","[30.0, 27.0, 28.0, 31.0, 29.0, 32.0, 33.0, 32....",33.0,0.000000,0.00,0.0


In [91]:
def extract_list_for_R(df, col):
    df = df.copy()
    df = df[['image_path', 'eye', 'dates', 'age', 'FP', 'FN', 'floss', col]]
    df = df.rename(columns={
        'image_path': 'id', 
        'dates': 'date', 
        'FP': 'fpr', 
        'FN': 'fnr', 
        'floss': 'fl'
    })
    df[[f'l{i}' for i in range(1, 55)]] = pd.DataFrame(df[col].tolist(), index= df.index)
    df = df.drop(columns=[col])
    df['time'] = '00:00:00'
    df['type'] = 'pwg'
    df['duration'] = '00:06:00'
    df = df[['id','eye','date','time','age','type','fpr','fnr','fl','duration'] + [f'l{i}' for i in range(1, 55)]]
    return df

df4R = extract_list_for_R(join_df, 'num_array')
df4R.to_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num.csv', index=False)

df4R = extract_list_for_R(join_df, 'pred_array')
df4R.to_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred.csv', index=False)

# 指标计算

## MAE

In [92]:
from sklearn.metrics import precision_recall_fscore_support

def get_metrics(exp_name, name):
    valid_cols = [f'l{i}' for i in range(1, 55)]
    valid_cols.remove('l26')
    valid_cols.remove('l35')
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred.csv')
    num = df_num[valid_cols].values
    pred = df_pred[valid_cols].values
    mae = mean_absolute_error(num, pred)
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num_pdp.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred_pdp.csv')
    
    class_mapping = {
        0: 1,
        0.005: 1,
        0.01: 1,
        0.02: 1,
        0.05: 1,
        0.95: 0,
        0.98: 0,
        0.99: 0,
        0.995: 0,
        1: 0,
    }
    for col in valid_cols:
        df_num[col] = df_num[col].map(class_mapping)
        df_pred[col] = df_pred[col].map(class_mapping)

    y_true = df_num[valid_cols].values.reshape(-1)
    y_pred = df_pred[valid_cols].values.reshape(-1)

    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]

    acc = (TP + TN) / (TP + FP + FN + TN)
    sensitivity = FP / (FP + TN)
    specificity = TN / (FP + TN)
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    
    df_num = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_num_pdp.csv')
    df_pred = pd.read_csv(f'{root_path}/{exp_name}/exp-pid/results/valid-loss/df4R_pred_pdp.csv')
    
    class_mapping = {
        0: 4,
        0.005: 4,
        0.01: 3,
        0.02: 2,
        0.05: 1,
        0.95: 0,
        0.98: 0,
        0.99: 0,
        0.995: 0,
        1: 0,
    }
    for col in valid_cols:
        df_num[col] = df_num[col].map(class_mapping)
        df_pred[col] = df_pred[col].map(class_mapping)

    y_true = df_num[valid_cols].values.reshape(-1)
    y_pred = df_pred[valid_cols].values.reshape(-1)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    
    _, _, fscores, _ = precision_recall_fscore_support(y_true, y_pred)
    return {
        'exp name': name,
        
        ('Regression', 'mae'): f'{mae:.3f}',
        
        ('2-class', 'accuracy'): f'{acc:.3f}',
        ('2-class', 'sensitivity'): f'{sensitivity:.3f}',
        ('2-class', 'specificity'): f'{specificity:.3f}',
        ('2-class', 'precision'): f'{precision:.3f}',
        ('2-class', 'recall'): f'{recall:.3f}',
        ('2-class', 'f1'): f'{F1:.3f}',
        
        ('5-class', '0.95-f1'): f'{fscores[0]:.3f}',
        ('5-class', '0.05-f1'): f'{fscores[1]:.3f}',
        ('5-class', '0.02-f1'): f'{fscores[2]:.3f}',
        ('5-class', '0.01-f1'): f'{fscores[3]:.3f}',
        ('5-class', '0.005-f1'): f'{fscores[4]:.3f}',
        
        ('5-class', 'macro-f1'): f'{macro_f1:.3f}',
        ('5-class', 'micro-f1'): f'{micro_f1:.3f}',
    }
df_stat = []
df_stat.append(get_metrics('0829/macula_num_r50_380', 'Macula'))
df_stat.append(get_metrics('0829/disc_num_r50_380', 'Disc'))
keys = [('Regression', 'mae'),
        ('2-class', 'accuracy'),
        ('2-class', 'sensitivity'),
        ('2-class', 'specificity'),
        ('2-class', 'precision'),
        ('2-class', 'recall'),
        ('2-class', 'f1'),
        
        ('5-class', '0.95-f1'),
        ('5-class', '0.05-f1'),
        ('5-class', '0.02-f1'),
        ('5-class', '0.01-f1'),
        ('5-class', '0.005-f1'),
        
        ('5-class', 'macro-f1'),
        ('5-class', 'micro-f1')]
df_stat = pd.DataFrame(df_stat, index=['Macula', 'Disc'], columns=pd.MultiIndex.from_tuples(keys))
# df_stat = df_stat.set_index('exp name')
print_df(df_stat)

[('Regression', 'mae'), ('2-class', 'accuracy'), ('2-class', 'sensitivity'), ('2-class', 'specificity'), ('2-class', 'precision'), ('2-class', 'recall'), ('2-class', 'f1'), ('5-class', '0.95-f1'), ('5-class', '0.05-f1'), ('5-class', '0.02-f1'), ('5-class', '0.01-f1'), ('5-class', '0.005-f1'), ('5-class', 'macro-f1'), ('5-class', 'micro-f1')]
2


Regression  2-class                                                  \
              mae accuracy sensitivity specificity precision recall     f1   
Macula      2.960    0.859       0.024       0.976     0.504  0.150  0.231   
Disc        3.106    0.850       0.033       0.967     0.625  0.276  0.383   

       5-class                                                     
       0.95-f1 0.05-f1 0.02-f1 0.01-f1 0.005-f1 macro-f1 micro-f1  
Macula   0.923   0.018   0.016   0.041    0.202    0.240    0.849  
Disc     0.915   0.051   0.023   0.033    0.397    0.284    0.834